In [ ]:
!pip install git+https://github.com/mimoralea/gym-walk#egg=gym-walk





  Cloning https://github.com/mimoralea/gym-walk to /tmp/pip-install-hnjywj6b/gym-walk_3b08142efd734c9a86d0f64aa88980e5
  Running command git clone --filter=blob:none --quiet https://github.com/mimoralea/gym-walk /tmp/pip-install-hnjywj6b/gym-walk_3b08142efd734c9a86d0f64aa88980e5
  Resolved https://github.com/mimoralea/gym-walk to commit b915b94cf2ad16f8833a1ad92ea94e88159279f5
  Preparing metadata (setup.py) ... done


In [ ]:
import warnings ; warnings.filterwarnings('ignore')

import gym, gym_walk
import numpy as np

import random
import warnings

warnings.filterwarnings('ignore', category=DeprecationWarning)
np.set_printoptions(suppress=True)
random.seed(123); np.random.seed(123)


In [ ]:
def print_policy(pi, P, action_symbols=('<', 'v', '>', '^'), n_cols=4, title='Policy:'):
    print(title)
    arrs = {k:v for k,v in enumerate(action_symbols)}
    for s in range(len(P)):
        a = pi(s)
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), arrs[a].rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")


In [ ]:
def print_state_value_function(V, P, n_cols=4, prec=3, title='State-value function:'):
    print(title)
    for s in range(len(P)):
        v = V[s]
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), '{}'.format(np.round(v, prec)).rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")


In [ ]:

def probability_success(env, pi, goal_state, n_episodes=100, max_steps=200):
    random.seed(123); np.random.seed(123) ; env.seed(123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        while not done and steps < max_steps:
            state, _, done, h = env.step(pi(state))
            steps += 1
        results.append(state == goal_state)
    return np.sum(results)/len(results)




In [ ]:
def mean_return(env, pi, n_episodes=100, max_steps=200):
    random.seed(123); np.random.seed(123) ; env.seed(123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        results.append(0.0)
        while not done and steps < max_steps:
            state, reward, done, _ = env.step(pi(state))
            results[-1] += reward
            steps += 1
    return np.mean(results)

In [ ]:
# Set up a custom FrozenLake environment
envdesc = [
    "SFFF",
    "FHFH",
    "FFFH",
    "HFFG"
]
env = gym.make('FrozenLake-v1', desc=envdesc, is_slippery=True)

# Reset environment and define the transition matrix
init_state = env.reset()
goal_state = 15  # Bottom-right in a 4x4 grid is state 15
P = env.env.P


In [ ]:
P

{0: {0: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  2: [(0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)],
  3: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 1: {0: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False)],
  2: [(0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  3: [(0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 2:

In [ ]:
def value_iteration(P, gamma=0.99, theta=1e-10):
    n_states = len(P)
    n_actions = len(P[0])
    V = np.zeros(n_states)

    while True:
        delta = 0
        for s in range(n_states):
            A = np.zeros(n_actions)
            for a in range(n_actions):
                for prob, next_state, reward, done in P[s][a]:
                    A[a] += prob * (reward + gamma * V[next_state])
            max_val = np.max(A)
            delta = max(delta, np.abs(V[s] - max_val))
            V[s] = max_val
        if delta < theta:
            break

    # Extract optimal policy
    def pi_opt(s):
        A = np.zeros(n_actions)
        for a in range(n_actions):
            for prob, next_state, reward, done in P[s][a]:
                A[a] += prob * (reward + gamma * V[next_state])
        return np.argmax(A)

    return V, pi_opt

In [ ]:

# Finding the optimal policy
V_best_v, pi_best_v = value_iteration(P, gamma=0.99)




In [ ]:
# Printing the policy
print("Name: NIRAUNJANA GAYATHRI G R  Register Number: 212222230096")
print('Optimal policy and state-value function (VI):')
print_policy(pi_best_v, P)




Name: NIRAUNJANA GAYATHRI G R  Register Number: 212222230096
Optimal policy and state-value function (VI):
Policy:
| 00      < | 01      ^ | 02      ^ | 03      ^ |
| 04      < |           | 06      < |           |
| 08      ^ | 09      v | 10      < |           |
|           | 13      > | 14      v |           |


In [ ]:
# printing the success rate and the mean return
print("Name: NIRAUNJANA GAYATHRI G R  Register Number: 212222230096")
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi_best_v, goal_state=goal_state)*100,
    mean_return(env, pi_best_v)))



Name: NIRAUNJANA GAYATHRI G R  Register Number: 212222230096
Reaches goal 69.00%. Obtains an average undiscounted return of 0.6900.


In [ ]:
# printing the state value function
print("Name: NIRAUNJANA GAYATHRI G R  Register Number: 212222230096")
print_state_value_function(V_best_v, P, prec=4)

Name: NIRAUNJANA GAYATHRI G R  Register Number: 212222230096
State-value function:
| 00  0.542 | 01 0.4988 | 02 0.4707 | 03 0.4569 |
| 04 0.5585 |           | 06 0.3583 |           |
| 08 0.5918 | 09 0.6431 | 10 0.6152 |           |
|           | 13 0.7417 | 14 0.8628 |           |
